In [1]:
#schillinger 

In [29]:
def schillinger_type_I(fraction):
    
    major = fraction[0]
    minor = fraction[1]
    seq_len = major*minor
    return_arr = [] 
    count = 1
    for i in range(1, seq_len):
        if i > 0 and i % major == 0 or i % minor == 0:
            return_arr.append(count)
            count = 1
        else:
            count += 1
    return_arr.append(count)
    return return_arr


fraction = [4,3]
schillinger_type_I(fraction)

[3, 1, 2, 2, 1, 3]